In [1]:
import pandas as pd
import numpy as np

In [2]:
# --- Configuration ---
file_path = r"C:\Users\daves\OneDrive\Pessoal\Artigo\Economic History Review\Code\Regression Analysis\Database for regression.xlsx"

In [3]:
# Define the list of columns to generate statistics for.
columns_to_analyze = [
    'Production',
    'WAP',
    'Total Tillage',
    'Annual Investment',
    'Capital Stock',
    'TFP'
]


# --- Data Loading ---
# Read the Excel file into a pandas DataFrame.
df = pd.read_excel(file_path)
df

,Codigo,Município,Emancipation,Latitude,Longitude,Mesorregião,NORDESTE MATO-GROSSENSE,NORTE MATO-GROSSENSE,SUDESTE MATO-GROSSENSE,SUDOESTE MATO-GROSSENSE,...,Permanently Tillage,Total Tillage,Productivity Land Unitilized,Planted Pastages,Natural Pastages,Total Pastages,Annual Investment,Capital Stock,Value of land,TFP
0,5100102,Acorizal,1953,-15.199491,-56.370149,CENTRO-SUL MATO-GROSSENSE,0,0,0,0,...,1001.210,4860.400,43920.42,0.0,0.0,0.0,3513.087399,40601.504150,7.092042e+04,2257.302943
1,5100201,Água Boa,1979,-14.057267,-52.161597,NORDESTE MATO-GROSSENSE,1,0,0,0,...,0.000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,NaN,0.000000
2,5100250,Alta Floresta,1976,-9.886111,-56.087222,NORTE MATO-GROSSENSE,0,1,0,0,...,0.000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,NaN,0.000000
3,5100300,Alto Araguaia,1938,-17.313918,-53.215700,SUDESTE MATO-GROSSENSE,0,0,1,0,...,165.159,8594.239,12506.92,0.0,0.0,0.0,16172.955573,80870.569671,3.328273e+05,-5951.712461
4,5100359,Alto Boa Vista,1991,-11.673900,-51.387800,NORDESTE MATO-GROSSENSE,1,0,0,0,...,0.000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,5108352,Vale de São Domingos,1999,-15.295776,-59.065426,SUDOESTE MATO-GROSSENSE,0,0,0,1,...,5135.000,9672.000,NaN,294523.0,20920.0,315443.0,4234.144382,9335.864004,3.561289e+05,-19447.416180
701,5108402,Várzea Grande,1948,-15.650000,-56.133333,CENTRO-SUL MATO-GROSSENSE,0,0,0,0,...,398.000,931.000,NaN,61534.0,137.0,61671.0,1624.682827,17968.507083,1.189117e+05,628.105788
702,5108501,Vera,1972,-12.305800,-55.316900,NORTE MATO-GROSSENSE,0,1,0,0,...,1111.000,91352.000,NaN,36302.0,20393.0,56695.0,12873.523293,34354.093501,6.848105e+05,26077.187105
703,5105507,Vila Bela da Santíssima Trindade,1752,-15.006425,-59.950917,SUDOESTE MATO-GROSSENSE,0,0,0,1,...,1230.000,7585.000,NaN,51323.0,24111.0,75434.0,29903.055309,53374.737638,2.905015e+06,5688.427890


In [4]:
# --- Data Transformation ---
# Rename columns to match the analysis and output requirements.
if 'PIA' in df.columns:
    df.rename(columns={'PIA': 'WAP', 'Ano': 'Year'}, inplace=True)
else:
    df.rename(columns={'Ano': 'Year'}, inplace=True)
    print("Warning: Original column 'PIA' not found to be renamed to 'WAP'.")


In [5]:

# --- Data Cleaning ---
# Replace any zero values in the specified columns with NaN (Not a Number).
for col in columns_to_analyze:
    if col in df.columns:
        df[col] = df[col].replace(0, np.nan)
    else:
        print(f"Warning: Column '{col}' was not found in the Excel file.")


In [6]:
# --- Data Aggregation and Analysis ---
# Group by 'Year' and calculate the specified aggregate statistics.
summary_table = df.groupby('Year')[columns_to_analyze].agg(
    ['min', 'max', 'std', 'mean', 'median']
)

In [7]:


# --- Prepare Data for Custom Excel Formatting ---
# Transpose the table to get variables as rows.
summary_table_transposed = summary_table.T
# Reset index to turn the multi-level index ('Variable', 'Descriptive Statistics') into columns.
summary_df_for_export = summary_table_transposed.reset_index()

In [8]:
# --- Export and Apply Advanced Styling ---
output_file_path = 'summary_statistics_by_year_formatted.xlsx'

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    # Write the DataFrame data without the default index or header.
    # We start writing at row 3 to leave space for our custom 2-row header.
    summary_df_for_export.to_excel(writer, sheet_name='Summary_Statistics', index=False, header=False, startrow=2)

    # --- Import styling tools from openpyxl ---
    from openpyxl.styles import Font, Border, Side, Alignment
    from openpyxl.utils import get_column_letter

    # --- Access the worksheet for styling ---
    worksheet = writer.sheets['Summary_Statistics']

    # --- Define Styles ---
    font_style = Font(name='Cambria', size=10)
    bold_font = Font(name='Cambria', size=10, bold=True)
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    center_align = Alignment(horizontal='center', vertical='center', wrap_text=True)

    # --- Create Custom Headers ---
    # Top-level headers in Row 1
    worksheet['A1'] = 'Variable'
    worksheet['B1'] = 'Descriptive Statistics'
    worksheet['C1'] = 'Year'

    # Merge header cells
    worksheet.merge_cells('A1:A2')
    worksheet.merge_cells('B1:B2')
    last_col_letter = get_column_letter(worksheet.max_column)
    worksheet.merge_cells(f'C1:{last_col_letter}1')

    # Second-level headers (the actual years) in Row 2
    year_headers = summary_table_transposed.columns
    for col_num, year in enumerate(year_headers, 3):
        cell = worksheet.cell(row=2, column=col_num)
        cell.value = year

    # --- Style Headers ---
    # **FIX:** Iterate through each row, and then each cell within that row.
    for row in worksheet["1:2"]:
        for cell in row:
            cell.font = bold_font
            cell.alignment = center_align

    # --- Merge and Style Variable Name Cells ---
    # The data starts at row 3. We step by 5 for each variable group.
    for row in range(3, worksheet.max_row + 1, 5):
        # Merge the 5 cells in the first column for each variable.
        worksheet.merge_cells(start_row=row, start_column=1, end_row=row + 4, end_column=1)
        cell = worksheet.cell(row, 1)
        cell.font = bold_font
        cell.alignment = center_align

    # --- Apply Borders, Centering, and Number Formatting to All Cells ---
    for row_cells in worksheet.iter_rows():
        for cell in row_cells:
            cell.border = thin_border
            # Apply standard font
            if not cell.font.bold:
                cell.font = font_style
            # Apply alignment to all cells with values
            if cell.value is not None:
                if cell.alignment.horizontal != 'center':
                    cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Apply number format to data cells only (not headers or labels)
            if cell.row > 2 and cell.column > 2 and isinstance(cell.value, (int, float)):
                cell.number_format = '0.00'


    # --- Auto-fit Column Widths ---
    worksheet.column_dimensions['A'].width = 20
    worksheet.column_dimensions['B'].width = 25


# --- Display Confirmation ---
print(f"--- Formatted summary table successfully exported to: {output_file_path} ---")

--- Formatted summary table successfully exported to: summary_statistics_by_year_formatted.xlsx ---
